In [2]:
import numpy as np
import collections
import time 

In [3]:
# Define function used to get the adjacency metrix from input graph file
def adjacent_matrix(filename):
    edge_node = np.loadtxt(filename, dtype = int)
    size = np.amax(edge_node)
    #print ('The number of vertices is:',size)
    adjmatrix = np.zeros((size+1,size+1))

    for i in range(edge_node.shape[0]):
        adjmatrix[edge_node[i,0], edge_node[i,1]] = 1
        adjmatrix[edge_node[i,1], edge_node[i,0]] = 1
    adjmatrix=adjmatrix[1:,1:]
    return adjmatrix 

In [4]:
# Get the adjacency matrix, R list(use to store all the vertices in the garph) 
# and the a dictionary use to store the degree of each vertice
def get_paramter(file):
    adjM= adjacent_matrix(file)
    R = []
    R = list (range (0,len(adjM)))
    ini_degree = {}
    for i in range (adjM.shape[0]):
        ini_degree[i]=np.count_nonzero(adjM[i])
    ini_degree= dict(sorted(ini_degree.items(), key = lambda x:(x[1])))
    return adjM, R, ini_degree

    

In [5]:
# get a nieghbor list for a vertice
def get_neig_list(vertice):
    neig_list = []
    for j in range(adjM.shape[1]):
        if adjM[vertice][j]==1:
            neig_list.append(j)
    return neig_list

In [6]:
# Find the vertice with minimum degree
def Find_R_min (R,degree):# pass the R list and degree dictionary
    dict= {}
    for vertex in R:
        dict[vertex]=degree[vertex]
        minval = min(dict.values())
    Rmin = [k for k, v in dict.items() if v==minval]
    return Rmin

In [7]:
# If multiple vertices have the same degree, 
# find the vertice whose neighbors have the minimum degree
def ex_deg (Rmin, initial_degree):
   
    exdeg_list=[]
    for i in Rmin:
        a=0
        for j in range (adjM.shape[1]):
            if adjM[i][j]==1:
                a += initial_degree[j]
        exdeg_list.append (a)
    return exdeg_list

In [8]:
#Using greedy coloring algorithm to sort the vertices 
# in a ascending order with respect to their color number
# This function is written according to the pseudo code from Tomita & Kameda paper
def NUMBER_SORT(R, No={}):
 
  # this part is Numbering  
    maxno=0 # initialize the max color number as 0
    n = len (R)
    C= collections.defaultdict(list) # create a dictionary used to store the vertices with same color number i
    C[0] = []# initialize the vertices with color 0 as empty 
    while len(R):
        p = R[0]  # pick the frist vertice in the R
        k = 0 # initialize color is 0
        p_neighbor = get_neig_list(p)
        while set(C[k]).intersection (set(p_neighbor)): # if p's color is same as its neighbor's color, p's color +1 
            k += 1
        if k > maxno: # if k greater than currect max color, updated max color number
            maxno=k
            C[maxno]=[]
        No[p]=k       # assgin p's color to k 
        C[k].append (p) # update color k list
        R.remove(p) # remove the p form R list 
    # this part is sorting
    R=[None]*n
    i=0
    for k in range (maxno+1):
        for j in range (len(C[k])):
            R[i]= C[k][j]
            i+=1
    return R, No

In [9]:
# Using the sorted R list and No dictionary to find a maximum clique 
# This function is written according to the pseudo code from Tomita & Kameda paper
def EXPAND (R, No):# pass the ordered R and No 
    global Q # store the vertices of a currect clique
    global Qmax # store the vertices of the largest clique found so far

    No=No.copy()# If without this copy, my code cannot work. I don't know the reason. 
                # I talked to Haoyang, he recommended me use a copy 
    while len(R):
        p=R[-1] # Start from Picking a vertice p with the max color number No
        if (len(Q)+No[p]+1) >len(Qmax):# because the color index starts from 0, so the number of color should plus 1
            Q.add (p)                  # add the p to the currect clique
            p_neighbor = get_neig_list(p)# get the neighbor list of p
            Rp = list ((set(R)).intersection(set(p_neighbor))) # find the new set of candidate vertices, by using intersection of currect R list and neighbors of p
            
            if len(Rp):
                Rp,No_prime =  NUMBER_SORT(Rp) # numbering and sorting based on the new candidate vertices,
                
                EXPAND(Rp,No_prime) # and expand
            elif len(Q)>len(Qmax): # If the currect clique is larger than the maximum clique, replace it
                Qmax = Q.copy()
                print('Max clique cardinality', len(Qmax))
            Q.remove(p) # remove p form Q and updated Q
        else:
            return
        R.remove(p)# remove p form R and updated R
    
    return Qmax
   

In [10]:
def start(list, value):
    for i in list:
        if i != value:
            return False
    return True

In [11]:
def MCR (file):
    starttime= time.perf_counter()
    global Q
    global Qmax
    Q = set()
    Qmax = set()

    global adjM
    adjM, R, ini_degree = get_paramter(file) #Get the adjacency matrix, R list and the a dictionary use to store the degree of each vertice

    n = len(adjM)# vertice number
    i = n-1 # index starts from 0, so i starts from n-1
    V = [None]*n  # initialize V as a empty list
    R = list (range (0,n)) # the R list used to store all the vertices
    Rmin = Find_R_min(R,ini_degree) # Find the vertices with minimum degree
    
    exdeg_list=[]
    degree = ini_degree.copy() # create a copy of degree dict, 
                               #becasue gegree is updating and still need to pass the original degree to ex_deg function
   
    while len (Rmin)!= len (R):
        if len (Rmin)>=2:       
            exdeg_list= ex_deg(Rmin, ini_degree)         #If multiple vertices have the same degree, 
                                                        # find the vertice whose neighbors have the minimum degree
            p_index = exdeg_list.index(min (exdeg_list))
            p = Rmin[p_index]
           
        else:
            p= Rmin[0]
        V[i]=p
        R.remove(p)     # remove p from R list
        i-=1
        for j in range(len(R)):           # after remove p, the degree of p neighbors should -1
            if R[j] in get_neig_list(p):
                degree[R[j]]-=1
        Rmin= Find_R_min(R,degree)        # update the Rmin based on the new R list
    Rmin, No = NUMBER_SORT (Rmin)         # numbering and sorting the new Rmin
    for i in range (len (Rmin)):
        V[i]= Rmin[i]
    m = max (No[q]for q in Rmin)+1        # define m as the max color number in Rmin
    mmax = len (Rmin)+max(degree.values())-m
    m+=1
    i= len (Rmin)
    while i <= mmax:
        if i > len(V):
            if start([degree[q] for q in Rmin], len(Rmin)-1):# to check if the maximum clique is the Rmin
                Qmax = Rmin  # updated Qmax
        No[V[i]] = m   #assign the V[i] color number as m
        m += 1
        i += 1    
    for i in range(mmax+1, len(V)):
        No[V[i]] = max(degree.values()) + 1        
    EXPAND(V,No)
    endtime= time.perf_counter()
    Usingtime = endtime - starttime
    print("The cardinality of maximum clique is: ",len(Qmax))

    print("Maximum clique is: ",Qmax)
    print ('Using Time{} seconds'.format (Usingtime))

In [12]:
MCR ('brock200_2.txt')


Max clique cardinality 8
Max clique cardinality 9
Max clique cardinality 10
Max clique cardinality 12
The cardinality of maximum clique is:  12
Maximum clique is:  {69, 134, 104, 47, 144, 148, 182, 119, 120, 54, 26, 157}
Using Time9.9830231 seconds


In [13]:
#MCR ('p_hat300_1.txt')

In [14]:
#MCR ('C125.9.txt')

In [15]:
#MCR('brock200_1.txt')

In [16]:
#print (Qmax)

In [17]:
#MCR ('p_hat300_3.txt')